In [1]:
%load_ext autoreload
%autoreload 2 
%reload_ext autoreload
%pylab inline
%matplotlib inline
import matplotlib.pyplot as plt

import sys; sys.path.insert(0, "../")
import sys; sys.path.insert(0, "../inverse_parametric_burgers/")

import numpy as np
from utils import *
from parametric_discovery_pinn import ParametricPINN, FinalParametricPINN
from madgrad import MADGRAD

from pde_diff import TrainSTRidge, FiniteDiff, print_pde
from robust_pde_diff import print_pde, RobustPCA, Robust_LRSTR, DLrSR
from parametric_pde_diff import TrainSGTRidge, create_groups
from pytorch_robust_pca import R_pca

from scipy.integrate import odeint
from numpy.fft import fft, ifft, fftfreq
from time import time

from pysr import pysr, best

fontsize = 20

Populating the interactive namespace from numpy and matplotlib


In [2]:
data = pickle_load("../parametric_pde_data/parametric_advection_diffusion.pkl")

x = data['x']; spatial_dims = x.shape[0]
t = data['t']; time_dims = t.shape[0]
Exact = data['u']

X, T = np.meshgrid(x, t)

X_star = np.hstack((to_column_vector(X), to_column_vector(T)))
u_star = to_column_vector(Exact.T)

# domain bounds
lb = X_star.min(axis=0)
ub = X_star.max(axis=0)

# Sampling training data points
N = 20000
training_idxs = sampling_from_rows(X_star, N, True)
X_train = X_star[training_idxs, :]
u_train = u_star[training_idxs, :]

# to_tensor
X_star = to_tensor(X_star, True)
u_star = to_tensor(u_star, False)
X_train = to_tensor(X_train, True)
u_train = to_tensor(u_train, False)
lb = to_tensor(lb, False)
ub = to_tensor(ub, False)

L = 5
u_true = -2*np.pi/L*np.sin(2*x*np.pi/L)
u_x_true = -1.5 + 1.0*np.cos(2*x*np.pi/L)
u_xx_true = 0.1*np.ones(spatial_dims)

eq_name = "ad"

Loaded from ../parametric_pde_data/parametric_advection_diffusion.pkl


In [3]:
pinn = ParametricPINN(n_funcs=3, scale=True, lb=lb, ub=ub, eq_name=eq_name)
pinn = load_weights(pinn, "./saved_path_inverse_parametric_ad/parametric_pinn_with_fg.pth")
model = nn.Sequential(pinn.preprocessor_net, pinn.pde_solver_net)
# 1.231026
pde_terms = ["u", "u_x", "u_xx"]
func_terms = ["-sin(1.2608153*x)", "cos(1.2608153*x)-1.5014627", "0.09376708"]
final_ad_pinn = FinalParametricPINN(model=model, pde_terms=pde_terms, 
                                        func_terms=func_terms, uncert=True, 
                                        scale=pinn.scale, lb=pinn.lb, ub=pinn.ub, 
                                        trainable_one=True
                                       )
del pinn

Loaded the model's weights properly


In [4]:
for i in range(3): print(see_params(final_ad_pinn.pdc.funcs[i]))

[tensor(-1.), tensor(1.2608)]
[tensor(-1.5015), tensor(1.2608)]
[tensor(0.0938)]


In [5]:
x, t = dimension_slicing(X_train)

In [6]:
def pcgrad_closure(return_list=False):
    global N, x, t, u_train
    losses = final_ad_pinn.loss(x, t, u_train)
    updated_grads = []
    
    for i in range(2):
        optimizer.zero_grad()
        losses[i].backward(retain_graph=True)

        g_task = []
        for param in final_ad_pinn.parameters():
            if param.grad is not None:
                g_task.append(Variable(param.grad.clone(), requires_grad=False))
            else:
                g_task.append(Variable(torch.zeros(param.shape), requires_grad=False))
        # appending the gradients from each task
        updated_grads.append(g_task)

    updated_grads = list(pcgrad.pc_grad_update(updated_grads))[0]
    for idx, param in enumerate(final_ad_pinn.parameters()):
        param.grad = (updated_grads[0][idx]+updated_grads[1][idx])
        
    if not return_list: return losses[0]+losses[1]
    else: return losses

In [7]:
def finetuning_closure():
    global N, x, t, u_train
    if torch.is_grad_enabled(): f_opt.zero_grad()
    # the solver network only consider the first N samples.
    mse_loss, pde_loss = final_ad_pinn.loss(x, t, u_train)
    loss = mse_loss + pde_loss
    if loss.requires_grad: loss.backward(retain_graph=False)
    return loss

In [8]:
optimizer = MADGRAD(final_ad_pinn.parameters(), lr=1e-6, momentum=0.9)
for i in range(150):
    final_ad_pinn.train()
    optimizer.step(pcgrad_closure)
    if i%10==0: print(pcgrad_closure(return_list=True))
        
f_opt = torch.optim.LBFGS(final_ad_pinn.parameters(), lr=1e-1, max_iter=500, max_eval=int(1.25*500), history_size=300, line_search_fn='strong_wolfe')
# final_ad_pinn.is_uncert = False
for i in range(100):
    f_opt.step(finetuning_closure)
    if i%10==0: print(finetuning_closure())

[tensor(0.6931, grad_fn=<AddBackward0>), tensor(0.9891, grad_fn=<AddBackward0>)]
[tensor(0.6929, grad_fn=<AddBackward0>), tensor(0.6995, grad_fn=<AddBackward0>)]
[tensor(0.6918, grad_fn=<AddBackward0>), tensor(0.6944, grad_fn=<AddBackward0>)]
[tensor(0.6907, grad_fn=<AddBackward0>), tensor(0.6922, grad_fn=<AddBackward0>)]
[tensor(0.6895, grad_fn=<AddBackward0>), tensor(0.6907, grad_fn=<AddBackward0>)]
[tensor(0.6883, grad_fn=<AddBackward0>), tensor(0.6893, grad_fn=<AddBackward0>)]
[tensor(0.6871, grad_fn=<AddBackward0>), tensor(0.6879, grad_fn=<AddBackward0>)]
[tensor(0.6859, grad_fn=<AddBackward0>), tensor(0.6866, grad_fn=<AddBackward0>)]
[tensor(0.6847, grad_fn=<AddBackward0>), tensor(0.6853, grad_fn=<AddBackward0>)]
[tensor(0.6834, grad_fn=<AddBackward0>), tensor(0.6840, grad_fn=<AddBackward0>)]
[tensor(0.6822, grad_fn=<AddBackward0>), tensor(0.6828, grad_fn=<AddBackward0>)]
[tensor(0.6810, grad_fn=<AddBackward0>), tensor(0.6815, grad_fn=<AddBackward0>)]
[tensor(0.6798, grad_fn=<Add

In [16]:
for i in range(3): print(see_params(final_ad_pinn.pdc.funcs[i]))

[tensor(-1.2554), tensor(1.2565)]
[tensor(-1.4998), tensor(1.2567)]
[tensor(0.0999)]


In [17]:
cc = 2.0*np.pi/5.0

In [18]:
[x.item() for x in final_ad_pinn.pdc.funcs.parameters()]

[-1.2553554773330688,
 1.2564677000045776,
 -1.4997742176055908,
 1.2567481994628906,
 0.09986680001020432]

In [19]:
errs = 100*np.array([abs(cc-1.2564677000045776)/cc, (1.5-1.4997742176055908)/1.5, abs(cc-1.2567481994628906)/cc, (0.1-0.09986680001020432)/0.1])
errs.mean(), errs.std()

(0.04264339677138114, 0.0523326488183105)

In [20]:
# save(final_ad_pinn, "./saved_path_inverse_parametric_ad/final_finetuned_pinn.pth")

In [ ]:
# mean, std
# (0.04264339677138114, 0.0523326488183105)